In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
from collections import namedtuple, defaultdict
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random

from multiprocessing.dummy import Pool as ThreadPool
import itertools

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential

from sklearn.model_selection import ParameterSampler

from thesis.utils.metrics import *

Using gpu device 0: TITAN X (Pascal) (CNMeM is disabled, cuDNN 5105)
/home/stud/shalaby/.virtualenv/thesis-env/local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
IS_SAMPLE = False

In [4]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234
WORD2VEC_SEED = 1234
NN_SEED = 1234

In [5]:
MIN_WORD_COUNT = 50
MIN_SIZE = 0
NUM_CORES = 16

In [6]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [7]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_MATRIX = "validation_matrix.pkl"
VALIDATION_DICT = "validation_dict.pkl"
TEST_MATRIX = "test_matrix.pkl"
TEST_DICT = "test_dict.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"
TYPE_CLASSIFIER= "{}_classifier.pkl"

TRAINING_DATA_MATRIX = "X.npy"
TRAINING_LABELS_MATRIX = "y_{}.npy"
VALIDATION_DATA_MATRIX = "Xv.npy"
VALIDATION_LABELS_MATRIX = "yv_{}.npy"

In [8]:
NN_PARAMETER_SEARCH_PREFIX = "standard_nn_{}_batch_{}_nn_parameter_searches.pkl"

In [9]:
root_location = "/mnt/data2/shalaby/"
exports_location = root_location + "exported_data/"

doc2vec_model_save_location = os.path.join("/mnt/data/shalaby/", "parameter_search_doc2vec_models_extended", "full")
nn_parameter_search_location = os.path.join(root_location, "nn_parameter_search_extended")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

#training_file = root_location + "docs_output.json"
training_file = root_location + 'docs_output.json'

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
valid_classes_file = exports_location + "valid_classes.pkl"
valid_subclasses_file = exports_location + "valid_subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
training_docs_list_file = exports_location + "extended_pv_training_docs_list.pkl"
validation_docs_list_file = exports_location + "extended_pv_validation_docs_list.pkl"
test_docs_list_file = exports_location + "extended_pv_test_docs_list.pkl"


training_doc_stats_file = exports_location + "extended_pv_training_doc_stats.pkl"
validation_doc_stats_file = exports_location + "extended_pv_validation_doc_stats.pkl"

preprocessed_location = root_location + "preprocessed_data/"

training_preprocessed_files_prefix = preprocessed_location + "extended_pv_training_docs_data_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "extended_pv_validation_docs_data_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "extended_pv_test_docs_data_preprocessed-"

word2vec_questions_file = result = root_location + 'tensorflow/word2vec/questions-words.txt'

In [10]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
valid_classes = pickle.load(open(valid_classes_file))
valid_subclasses = pickle.load(open(valid_subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 15.4 s, sys: 692 ms, total: 16.1 s
Wall time: 16.1 s


In [11]:
len(training_docs_list)

120156

In [12]:
len(validation_docs_list)

29675

In [13]:
len(test_docs_list)

37771

In [14]:
def ensure_disk_location_exists(location):
    if not os.path.exists(location):
        os.makedirs(location)

In [15]:
VALIDATION_MINI_BATCH_SIZE = 10000
def get_extended_docs_with_inference(doc2vec_model, doc_classification_map, classifications,
                                           docs_list, file_to_write, preprocessed_files_prefix):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)


    one_hot_encoder = OneHotEncoder(classifications)
    classifications_set = set(classifications)
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_labels = []
        inference_vectors_matrix = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
        for i, doc_id in enumerate(docs_list):
            curr_doc_labels = set(doc_classification_map[doc_id]) & classifications_set
            inference_labels.append(one_hot_encoder.get_label_vector(curr_doc_labels))
            if i % 100000 == 0:
                info("Finished {} in validation loading".format(i))
        inference_labels = np.array(inference_labels, dtype=np.int8)
    else:
        inference_documents_reps = {}
        inference_vectors = []
        inference_labels = []
        info("===== Getting vectors with inference")

        # Multi-threaded inference
        inference_docs_iterator = ExtendedPVDocumentBatchGenerator(preprocessed_files_prefix, batch_size=None)
        generator_func = inference_docs_iterator.__iter__()
        pool = ThreadPool(NUM_CORES)
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = VALIDATION_MINI_BATCH_SIZE
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {} tags".format(str(inference_docs_iterator.curr_index + 1)))
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break

        # create matrix for the inferred vectors
        for doc_id in docs_list:
            inference_vectors.append(inference_documents_reps[doc_id])
            curr_doc_labels = set(doc_classification_map[doc_id]) & classifications_set
            inference_labels.append(one_hot_encoder.get_label_vector(curr_doc_labels))
        inference_vectors_matrix = np.array(inference_vectors)
        inference_labels = np.array(inference_labels, dtype=np.int8)
        pickle.dump(inference_vectors_matrix,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_vectors_matrix, inference_labels

In [16]:
class OneHotEncoder():
    
    def __init__(self, classifications):
        self.classifications = classifications
        self.one_hot_indices = {}

        # convert character classifications to bit vectors
        for i, clssf in enumerate(classifications):
            bits = [0] * len(classifications)
            bits[i] = 1
            self.one_hot_indices[clssf] = i
    
    def get_label_vector(self, labels):
        """
        classes: array of string with the classes assigned to the instance
        """
        output_vector = [0] * len(self.classifications)
        for label in labels:
            index = self.one_hot_indices[label]
            output_vector[index] = 1
            
        return output_vector


In [17]:
class DocsOnlyExtendedPVDocumentBatchGenerator(object):
    def __init__(self, filename_prefix, batch_size=10000 ):
        """
        batch_size cant be > 10,000 due to a limitation in doc2vec training, 
        None means no batching (only use for inference)
        """
        assert batch_size <= 10000 or batch_size is None
        self.filename_prefix = filename_prefix
        self.curr_lines = []
        self.curr_docids = []
        self.batch_size = batch_size
        self.curr_index = 0
        self.batch_end = -1
    def load_new_batch_in_memory(self):
        del self.curr_lines, self.curr_docids
        self.curr_lines, self.curr_docids = [], []
        info("Loading new batch for index: {}".format(self.curr_index))
        true_docs_count = 0
        try:
            with open(self.filename_prefix + str(self.curr_index)) as preproc_file:
                for line in preproc_file:
                    line_array = line.split(" ")
                    if is_real_doc(doc_id):
                        doc_id = line_array[0]
                        doc_tokens = line_array[1:]
                        self.curr_lines.append(doc_tokens)
                        self.curr_docids.append(doc_id)
                        true_docs_count+= 1
            self.batch_end = self.curr_index + true_docs_count - 1 
            info(true_docs_count)
            info("Finished loading new batch")
        except IOError:
            info("No more batches to load, exiting at index: {}".format(self.curr_index))
            raise StopIteration()
    def __iter__(self):
        while True:
            if self.curr_index > self.batch_end:
                self.load_new_batch_in_memory()
            for (doc_id, tokens) in zip(self.curr_docids, self.curr_lines):
                if self.batch_size is not None:
                    curr_batch_iter = 0
                    # divide the document to batches according to the batch size
                    while curr_batch_iter < len(tokens):
                        yield LabeledSentence(words=tokens[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id])
                        curr_batch_iter += self.batch_size
                else:
                    yield doc_id, tokens
                self.curr_index += 1
                if self.curr_index % 1000 == 0:
                    info("New Doc: {:5}, Batch End: {:5}".format(self.curr_index, self.batch_end))

def is_real_doc(doc_id):
    return doc_id.find("_") == -1

## Get Document, Paragraph and Sentence Stats

In [18]:
class DocumentsStatsGenerator(object):
    def __init__(self, filename_prefix):
        self.filename_prefix = filename_prefix
        self.docids = []
        self.doc_paragraphs = defaultdict(list)
        self.doc_sentences = defaultdict(list)
        self.curr_doc_index = 0
        self.batch_end = -1
    def load_new_batch_in_memory(self):
        info("Loading new batch for index: {}".format(self.curr_doc_index))
        true_docs_count = 0
        try:
            with open(self.filename_prefix + str(self.curr_doc_index)) as preproc_file:
                for line in preproc_file:
                    line_array = line.split(" ", 1)
                    entity_id = line_array[0].strip()
                    if self.is_doc(entity_id):
                        self.docids.append(entity_id)
                        true_docs_count+= 1
                    elif self.is_paragraph(entity_id):
                        self.doc_paragraphs[self.get_doc_id(entity_id)].append(entity_id)
                    elif self.is_sentence(entity_id):
                        self.doc_sentences[self.get_doc_id(entity_id)].append(entity_id)
            self.batch_end = self.curr_doc_index + true_docs_count - 1 
            info("Finished loading new batch of {} documents".format(true_docs_count))
        except IOError:
            info("No more batches to load, exiting at index: {}".format(self.curr_doc_index))
            raise StopIteration()
    def get_stats(self):
        try:
            while True:
                if self.curr_doc_index > self.batch_end:
                    self.load_new_batch_in_memory()
                self.curr_doc_index = self.batch_end + 1
        except StopIteration:
            pass
            
    def get_doc_id(self, entity_id):
        return entity_id.split("_")[0]
    def get_entity_parts(self, entity_id):
        return entity_id.split("_")
    def is_doc(self, entity_id):
        parts = self.get_entity_parts(entity_id)
        if len(parts) == 1:
            return True
        return False
    def is_paragraph(self, entity_id):
        parts = self.get_entity_parts(entity_id)
        if len(parts) == 2:
            return True
        return False
    def is_sentence(self, entity_id):
        parts = self.get_entity_parts(entity_id)
        if len(parts) == 3:
            return True
        return False

In [19]:
def get_doc_vector(entity_id):
    if DOC2VEC_MMAP:
        normal_array = []
        normal_array[:] = doc2vec_model.docvecs[entity_id][:]
        return normal_array
    else:
        return doc2vec_model.docvecs[entity_id]

def data_generator(doc_stats, doc_id):
    yield get_doc_vector(doc_id)
    for paragraph_id in doc_stats.doc_paragraphs[doc_id]:
        yield get_doc_vector(paragraph_id)
    for sentence_id in doc_stats.doc_sentences[doc_id]:
        yield get_doc_vector(sentence_id)
    while True:
        yield ZERO_VECTOR


def validation_data_generator(doc_stats, validation_dict, doc_id):
    yield validation_dict[doc_id]
    for paragraph_id in doc_stats.doc_paragraphs[doc_id]:
        yield validation_dict[paragraph_id]
    for sentence_id in doc_stats.doc_sentences[doc_id]:
        yield validation_dict[sentence_id]
    while True:
        yield ZERO_VECTOR
        

In [20]:
def get_training_data(doc2vec_model, classifications):
    one_hot_encoder = OneHotEncoder(classifications)
    classifications_set = set(classifications)
    training_data = []
    training_labels_mat = np.zeros((len(training_docs_list), len(classifications)), dtype=np.int8)
#     training_data_mat = np.zeros((len(training_docs_list), DOC2VEC_SIZE), dtype=np.float32)
    for i,doc_id in enumerate(training_docs_list):
        # converting from memmap to a normal array
#         normal_array = []
#         normal_array[:] = doc2vec_model.docvecs[doc_id][:]
        normal_array = doc2vec_model.docvecs[doc_id]
        training_data.append(normal_array)
#         eligible_classifications = [clssf for clssf in doc_classification_map[doc_id] if clssf in classifications]
        eligible_classifications = set(doc_classification_map[doc_id]) & classifications_set
        training_labels_mat[i][:] = one_hot_encoder.get_label_vector(eligible_classifications)
        if i % 100000 == 0:
            info("Finished {} in training".format(i))
    info("doing matrix creation")
    training_data_mat = np.array(training_data)
#     training_labels_mat = np.array(training_labels, dtype=np.int8)
    del training_data
    return training_data_mat, training_labels_mat

In [21]:
def get_validation_data(validation_dict, classifications, docs_list):
    
    one_hot_encoder = OneHotEncoder(classifications)
    classifications_set = set(classifications)
    inference_vectors = []
    inference_labels = []
    info("===== Getting vectors with inference")

    # create matrix for the inferred vectors
    for doc_id in docs_list:
        inference_vectors.append(validation_dict[doc_id])
        curr_doc_labels = set(doc_classification_map[doc_id]) & classifications_set
        inference_labels.append(one_hot_encoder.get_label_vector(curr_doc_labels))
    inference_vectors_matrix = np.array(inference_vectors)
    inference_labels = np.array(inference_labels, dtype=np.int8)

    return inference_vectors_matrix, inference_labels

## Actual Training, validation and Metrics Loop

In [22]:
from keras.layers import Input, Masking
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers.pooling import GlobalAveragePooling1D
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras

In [23]:
classifications = sections
classifications_type = 'sections'
classifier_file = TYPE_CLASSIFIER.format(classifications_type)

In [24]:
VALIDATION_METRICS_FILENAME= '{}_validation_metrics.pkl'.format(classifications_type)
TRAINING_METRICS_FILENAME = '{}_training_metrics.pkl'.format(classifications_type)

#### Load the Doc2vec model

In [25]:
DOC2VEC_SIZE = 200
DOC2VEC_WINDOW = 2
DOC2VEC_MAX_VOCAB_SIZE = None
DOC2VEC_SAMPLE = 1e-3
DOC2VEC_TYPE = 1
DOC2VEC_HIERARCHICAL_SAMPLE = 0
DOC2VEC_NEGATIVE_SAMPLE_SIZE = 10
DOC2VEC_CONCAT = 0
DOC2VEC_MEAN = 1
DOC2VEC_TRAIN_WORDS = 0
DOC2VEC_EPOCHS = 1 # we do our training manually one epoch at a time
DOC2VEC_MAX_EPOCHS = 8
REPORT_DELAY = 20 # report the progress every x seconds
REPORT_VOCAB_PROGRESS = 100000 # report vocab progress every x documents

DOC2VEC_MMAP = 'r'
# DOC2VEC_MMAP = None

ZERO_VECTOR = [0] * DOC2VEC_SIZE

In [26]:
placeholder_model_name = 'doc2vec_size_{}_w_{}_type_{}_concat_{}_mean_{}_trainwords_{}_hs_{}_neg_{}_vocabsize_{}'.format(DOC2VEC_SIZE, 
                                                                DOC2VEC_WINDOW, 
                                                                'dm' if DOC2VEC_TYPE == 1 else 'pv-dbow',
                                                                DOC2VEC_CONCAT, DOC2VEC_MEAN,
                                                                DOC2VEC_TRAIN_WORDS,
                                                                DOC2VEC_HIERARCHICAL_SAMPLE,DOC2VEC_NEGATIVE_SAMPLE_SIZE,
                                                                str(DOC2VEC_MAX_VOCAB_SIZE))
GLOBAL_VARS.DOC2VEC_MODEL_NAME = placeholder_model_name
placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")

epoch = 8

GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
doc2vec_model = None
print GLOBAL_VARS.MODEL_NAME

doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8


In [27]:
%%time
print os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)
if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
    doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX), mmap=DOC2VEC_MMAP)
    doc2vec_model.workers = NUM_CORES
    GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
else:
    info("Couldnt find the doc2vec model with epoch {}".format(epoch))
    raise Exception()

2017-03-20 20:29:32,661 : INFO : loading Doc2Vec object from /mnt/data/shalaby/parameter_search_doc2vec_models_extended/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model


/mnt/data/shalaby/parameter_search_doc2vec_models_extended/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model


2017-03-20 20:32:10,419 : INFO : loading docvecs recursively from /mnt/data/shalaby/parameter_search_doc2vec_models_extended/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.docvecs.* with mmap=r
2017-03-20 20:32:10,420 : INFO : loading doctag_syn0 from /mnt/data/shalaby/parameter_search_doc2vec_models_extended/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.docvecs.doctag_syn0.npy with mmap=r
2017-03-20 20:32:10,476 : INFO : loading doctag_syn0_lockf from /mnt/data/shalaby/parameter_search_doc2vec_models_extended/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.docvecs.doctag_syn0_lockf.npy with mmap=r
2017-03-20 20:32:10,487 : INFO : loading syn1neg from /mnt/data/shalaby/parameter_search_doc2vec_models_extended/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model.syn1ne

CPU times: user 1min 59s, sys: 25.9 s, total: 2min 25s
Wall time: 2min 38s


### Create/Load Training Document Stats

In [28]:
%%time
if not os.path.exists(training_doc_stats_file):
    info("Creating Training Document Stats")
    doc_stats = DocumentsStatsGenerator(training_preprocessed_files_prefix)
    doc_stats.get_stats()
    pickle.dump(doc_stats, open(training_doc_stats_file, "w"))
else:
    info("Loading Training Document Stats")
    doc_stats = pickle.load(open(training_doc_stats_file, "r"))

2017-03-20 20:32:11,159 : INFO : Loading Training Document Stats


CPU times: user 59.3 s, sys: 5.18 s, total: 1min 4s
Wall time: 1min 4s


In [31]:
MAX_PARAGRAPHS = int(np.percentile([len(doc_stats.doc_paragraphs[d]) for d in doc_stats.docids], 50))
print "Paragraphs: {}".format(MAX_PARAGRAPHS)
MAX_SENTENCES = int(np.percentile([len(doc_stats.doc_sentences[d]) for d in doc_stats.docids], 50))
print "Sentences: {}".format(MAX_SENTENCES)
# +1 is for the document vector
MAX_SIZE = MAX_PARAGRAPHS + MAX_SENTENCES + 1
print "Max Size: {}".format(MAX_SIZE)

Paragraphs: 47
Sentences: 196
Max Size: 244


In [32]:
MAX_SIZE = 244

In [33]:
max([len(doc_stats.doc_sentences[d]) for d in doc_stats.docids])

20368

In [34]:
max([len(doc_stats.doc_paragraphs[d]) for d in doc_stats.docids])

4686

### Get Training Data Matrices

In [29]:
%%time
X, y = get_training_data(doc2vec_model, classifications)

2017-03-20 20:33:15,616 : INFO : Finished 0 in training
2017-03-20 20:33:16,839 : INFO : Finished 100000 in training
2017-03-20 20:33:17,030 : INFO : doing matrix creation


CPU times: user 3.2 s, sys: 7.12 s, total: 10.3 s
Wall time: 3min 26s


In [30]:
import sys
print sys.getsizeof(X)
print X.shape

96124912
(120156, 200)


## Create Validation Matrix

In [31]:
validation_dict = None

Load Validation Dict

In [32]:
%%time
validation_dict = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, VALIDATION_DICT)))

CPU times: user 17min 27s, sys: 37.8 s, total: 18min 4s
Wall time: 18min 8s


### Get Validation Data Matrices

In [33]:
%%time
Xv, yv = get_validation_data(validation_dict, classifications, validation_docs_list)

2017-03-20 20:55:11,785 : INFO : ===== Getting vectors with inference


CPU times: user 280 ms, sys: 512 ms, total: 792 ms
Wall time: 758 ms


# Actual Training

In [34]:
def create_keras_nn_model(input_size, output_size, 
                          first_hidden_layer_size, first_hidden_layer_activation, 
                          second_hidden_layer_size, second_hidden_layer_activation, 
                          input_dropout_do, hidden_dropout_do, second_hidden_dropout_do=False):
    
    doc_input = Input(shape=(input_size,), name='doc_input')
    if input_dropout_do:
        hidden = Dropout(0.7)(doc_input)
    hidden = Dense(first_hidden_layer_size, activation=first_hidden_layer_activation, 
                   name='hidden_layer_{}'.format(first_hidden_layer_activation))(doc_input if not input_dropout_do else hidden)
    if hidden_dropout_do:
        hidden = Dropout(0.5)(hidden)
    if second_hidden_layer_size is not None:
        hidden = Dense(second_hidden_layer_size, activation=second_hidden_layer_activation, 
                       name='hidden_layer2_{}'.format(second_hidden_layer_activation))(hidden)
    if second_hidden_dropout_do:
        hidden = Dropout(0.5)(hidden)
    softmax_output = Dense(output_size, activation='sigmoid', name='softmax_output')(hidden)

    model = Model(input=doc_input, output=softmax_output)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')
    
    return model

In [36]:
early_stopper_deltas = {
    'sections': 0.0001,
    'classes': 0.00001,
    'subclasses': 0.00001
}
early_stopper_patience = {
    'sections': 10,
    'classes': 10,
    'subclasses': 10
}
epochs_before_validation = {
    'sections': 10,
    'classes': 10,
    'subclasses': 10
}

In [37]:
NN_OUTPUT_NEURONS = len(classifications)

EARLY_STOPPER_MIN_DELTA = early_stopper_deltas[classifications_type]
EARLY_STOPPER_PATIENCE = early_stopper_patience[classifications_type]

NN_MAX_EPOCHS = 100
NN_RANDOM_SEARCH_BUDGET = 30
NN_PARAM_SAMPLE_SEED = 1234

NN_BATCH_SIZE = 2048

MODEL_VERBOSITY = 0

to_skip = []

load_existing_results = True
save_results = True


first_hidden_layer_sizes = [100,200,500]
# first_hidden_layer_sizes = [1000,2000]
# second_hidden_layer_sizes = [1000,2000,3000,4000]
second_hidden_layer_sizes = [None,500,1000,2000]
first_hidden_layer_activations = ['relu','sigmoid', 'tanh']
second_hidden_layer_activations = ['relu','sigmoid', 'tanh']
# first_hidden_layer_activations = ['relu']
# second_hidden_layer_activations = ['relu']
# input_dropout_options = [False, True]
# hidden_dropout_options = [False, True]
input_dropout_options = [False]
hidden_dropout_options = [True]
second_hidden_dropout_options = [False]


np.random.seed(NN_SEED)


# # Uncomment for Specific Configuration
# NN_RANDOM_SEARCH_BUDGET = 1
# first_hidden_layer_sizes = [100]
# second_hidden_layer_sizes = [1000]
# first_hidden_layer_activations = ['tanh']
# second_hidden_layer_activations = ['relu']
# input_dropout_options = [False]
# hidden_dropout_options = [True]
# second_hidden_dropout_options = [False]

In [38]:
class MetricsCallback(keras.callbacks.Callback):
    
    EPOCHS_BEFORE_VALIDATION = epochs_before_validation[classifications_type]
    
    def on_train_begin(self, logs={}):
        self.epoch_index = 0
        self.val_loss_reductions = 0
        self.metrics_dict = {}
        self.best_val_loss = np.iinfo(np.int32).max
        self.best_weights = None
        self.best_validation_metrics = None
    def on_epoch_end(self, epoch, logs={}):
        self.epoch_index += 1
        if logs['val_loss'] < self.best_val_loss:
            self.val_loss_reductions += 1
            self.best_val_loss = logs['val_loss']
            self.best_weights = self.model.get_weights()
            #print '\r    \r' # to remove the previous line of verbose output of model fit
            #time.sleep(0.2)
            info('Found lower val loss for epoch {} => {}'.format(self.epoch_index, round(logs['val_loss'], 5)))
            if self.val_loss_reductions % MetricsCallback.EPOCHS_BEFORE_VALIDATION == 0:
                
                info('Validation Loss Reduced {} times'.format(self.val_loss_reductions))
                info('Evaluating on Validation Data')
                yvp = self.model.predict(Xv)
                yvp_binary = get_binary_0_5(yvp)
                info('Generating Validation Metrics')
                validation_metrics = get_metrics(yv, yvp, yvp_binary)
                print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
                    validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
                    validation_metrics['f1_micro'], validation_metrics['f1_macro'])
                self.metrics_dict[self.epoch_index] = validation_metrics
#                 self.best_validation_metrics = validation_metrics

In [39]:
param_sampler = ParameterSampler({
    'first_hidden_layer_size':first_hidden_layer_sizes,
    'first_hidden_layer_activation':first_hidden_layer_activations,
    'second_hidden_layer_size':second_hidden_layer_sizes,
    'second_hidden_layer_activation':second_hidden_layer_activations,
    'input_dropout':input_dropout_options,
    'hidden_dropout':hidden_dropout_options,
    'second_hidden_dropout':second_hidden_dropout_options
}, n_iter=NN_RANDOM_SEARCH_BUDGET, random_state=NN_PARAM_SAMPLE_SEED)

param_results_dict = {}
if load_existing_results:
    param_results_path = os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                       NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE)))
    if os.path.exists(param_results_path):
        info('Loading Previous results in {}'.format(param_results_path))
        param_results_dict = pickle.load(open(param_results_path))
    else:
        info('No Previous results exist in {}'.format(param_results_path))

# create nn parameter search directory
if not os.path.exists(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME)):
    os.makedirs(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME))
        
for parameters in param_sampler:
    start_time = time.time()
    first_hidden_layer_size = parameters['first_hidden_layer_size']
    first_hidden_layer_activation = parameters['first_hidden_layer_activation']
    second_hidden_layer_size = parameters['second_hidden_layer_size']
    second_hidden_layer_activation = parameters['second_hidden_layer_activation']
    input_dropout_do = parameters['input_dropout']
    hidden_dropout_do = parameters['hidden_dropout']
    second_hidden_dropout_do = parameters['second_hidden_dropout']

    GLOBAL_VARS.NN_MODEL_NAME = 'nn_1st-size_{}_1st-act_{}_2nd-size_{}_2nd-act_{}_in-drop_{}_hid-drop_{}'.format(
        first_hidden_layer_size, first_hidden_layer_activation, second_hidden_layer_size, 
        second_hidden_layer_activation, input_dropout_do, hidden_dropout_do
    )
    if second_hidden_dropout_do:
        GLOBAL_VARS.NN_MODEL_NAME = GLOBAL_VARS.NN_MODEL_NAME + '_2nd-hid-drop_{}'.format(str(second_hidden_dropout_do))

    if GLOBAL_VARS.NN_MODEL_NAME in param_results_dict.keys() or GLOBAL_VARS.NN_MODEL_NAME in to_skip:
        print "skipping: {}".format(GLOBAL_VARS.NN_MODEL_NAME)
        continue
#         if first_hidden_layer_size < DOC2VEC_SIZE or second_hidden_layer_size < NN_OUTPUT_NEURONS:
#             print "skipping: {} due to 1st layer size {} < {} or 2nd layer size {} < {}".format(GLOBAL_VARS.NN_MODEL_NAME,
#                                                                                                 first_hidden_layer_size, DOC2VEC_SIZE, 
#                                                                                                 second_hidden_layer_size, NN_OUTPUT_NEURONS)
#             continue


    info('***************************************************************************************')
    info(GLOBAL_VARS.NN_MODEL_NAME)

    model = create_keras_nn_model(DOC2VEC_SIZE, NN_OUTPUT_NEURONS, 
                                  first_hidden_layer_size, first_hidden_layer_activation, 
                                  second_hidden_layer_size, second_hidden_layer_activation, 
                                  input_dropout_do, hidden_dropout_do, second_hidden_dropout_do)
    model.summary()

    early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=EARLY_STOPPER_MIN_DELTA, \
                                                  patience=EARLY_STOPPER_PATIENCE, verbose=1, mode='auto')
    metrics_callback = MetricsCallback()

    # Model Fitting
    %time history = model.fit(x=X, y=y, validation_data=(Xv,yv), batch_size=NN_BATCH_SIZE, \
                              nb_epoch=NN_MAX_EPOCHS, verbose=MODEL_VERBOSITY, callbacks=[early_stopper, metrics_callback])


    # using the recorded weights of the best recorded validation loss
    last_model_weights = model.get_weights()
    info('Evaluating on Validation Data using saved best weights')
    model.set_weights(metrics_callback.best_weights)
    yvp = model.predict(Xv)
    yvp_binary = get_binary_0_5(yvp)
    #print yvp
    info('Generating Validation Metrics')
    validation_metrics = get_metrics(yv, yvp, yvp_binary)
    print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
        validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
        validation_metrics['f1_micro'], validation_metrics['f1_macro'])
    best_validation_metrics = validation_metrics
    time.sleep(0.2)

    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME] = dict()
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_validation_metrics'] = best_validation_metrics
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['epochs'] = len(history.history['val_loss'])
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_weights'] = metrics_callback.best_weights
#         param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['last_weights'] = last_model_weights

    duration = time.time() - start_time
    param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['duration'] =  duration

    del history, last_model_weights, metrics_callback

if save_results:
    pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                                                   NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))


2017-03-20 21:06:26,707 : INFO : No Previous results exist in /mnt/data2/shalaby/nn_parameter_search_extended/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/standard_nn_sections_batch_2048_nn_parameter_searches.pkl
2017-03-20 21:06:26,710 : INFO : ***************************************************************************************
2017-03-20 21:06:26,711 : INFO : nn_1st-size_500_1st-act_relu_2nd-size_500_2nd-act_tanh_in-drop_False_hid-drop_True


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
doc_input (InputLayer)           (None, 200)           0                                            
____________________________________________________________________________________________________
hidden_layer_relu (Dense)        (None, 500)           100500      doc_input[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 500)           0           hidden_layer_relu[0][0]          
____________________________________________________________________________________________________
hidden_layer2_tanh (Dense)       (None, 500)           250500      dropout_1[0][0]                  
___________________________________________________________________________________________

2017-03-20 21:06:32,603 : INFO : Found lower val loss for epoch 1 => 0.30382
2017-03-20 21:06:33,621 : INFO : Found lower val loss for epoch 2 => 0.28485
2017-03-20 21:06:34,610 : INFO : Found lower val loss for epoch 3 => 0.26726
2017-03-20 21:06:35,958 : INFO : Found lower val loss for epoch 4 => 0.26076
2017-03-20 21:06:37,984 : INFO : Found lower val loss for epoch 6 => 0.25115
2017-03-20 21:06:39,682 : INFO : Found lower val loss for epoch 8 => 0.25052
2017-03-20 21:06:40,605 : INFO : Found lower val loss for epoch 9 => 0.248
2017-03-20 21:06:51,563 : INFO : Evaluating on Validation Data using saved best weights


Epoch 00019: early stopping
CPU times: user 8 s, sys: 10.4 s, total: 18.4 s
Wall time: 21.7 s


2017-03-20 21:06:52,328 : INFO : Generating Validation Metrics


...

****** Validation Metrics: Cov Err: 1.889 | Top 3: 0.901 | Top 5: 0.979 | F1 Micro: 0.570 | F1 Macro: 0.506


2017-03-20 21:34:48,807 : INFO : Found lower val loss for epoch 15 => 0.26088
2017-03-20 21:34:49,849 : INFO : Found lower val loss for epoch 16 => 0.25831
2017-03-20 21:34:50,792 : INFO : Found lower val loss for epoch 17 => 0.25704
2017-03-20 21:34:53,569 : INFO : Found lower val loss for epoch 20 => 0.2565
2017-03-20 21:34:54,504 : INFO : Found lower val loss for epoch 21 => 0.25478
2017-03-20 21:34:55,351 : INFO : Found lower val loss for epoch 22 => 0.25311
2017-03-20 21:35:00,180 : INFO : Found lower val loss for epoch 27 => 0.25243
2017-03-20 21:35:11,567 : INFO : Evaluating on Validation Data using saved best weights


Epoch 00037: early stopping
CPU times: user 15.4 s, sys: 16.7 s, total: 32 s
Wall time: 40.5 s


2017-03-20 21:35:12,066 : INFO : Generating Validation Metrics


****** Validation Metrics: Cov Err: 1.793 | Top 3: 0.917 | Top 5: 0.983 | F1 Micro: 0.610 | F1 Macro: 0.554


In [71]:
%%time
%time yp = model.predict(X)
%time yp_binary = get_binary_0_5(yp)
#print yvp
info('Generating Training Metrics')
training_metrics = get_metrics(y, yp, yp_binary)
print "****** Training Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
    training_metrics['coverage_error'], training_metrics['top_3'], training_metrics['top_5'], 
    training_metrics['f1_micro'], training_metrics['f1_macro'])

CPU times: user 1.86 s, sys: 620 ms, total: 2.48 s
Wall time: 2.49 s


2017-03-13 00:01:30,344 : INFO : Generating Training Metrics


CPU times: user 6.32 s, sys: 3.08 s, total: 9.4 s
Wall time: 9.47 s
****** Training Metrics: Cov Err: 11.998 | Top 3: 0.540 | Top 5: 0.636 | F1 Micro: 0.315 | F1 Macro: 0.068
CPU times: user 21.3 s, sys: 7.25 s, total: 28.6 s
Wall time: 28.6 s


In [ ]:
del validation_dict

In [71]:
model.fit(x=X, y=y, batch_size=NN_BATCH_SIZE, \
                              nb_epoch=10, verbose=MODEL_VERBOSITY)

Epoch 1/10
20000/20000 [==============================] - 44s - loss: 0.2937    
Epoch 2/10
20000/20000 [==============================] - 44s - loss: 0.2882    
Epoch 3/10
20000/20000 [==============================] - 44s - loss: 0.2843    
Epoch 4/10
20000/20000 [==============================] - 44s - loss: 0.2808    
Epoch 5/10
20000/20000 [==============================] - 44s - loss: 0.2779    
Epoch 6/10
20000/20000 [==============================] - 44s - loss: 0.2743    
Epoch 7/10
20000/20000 [==============================] - 44s - loss: 0.2729    
Epoch 8/10
20000/20000 [==============================] - 44s - loss: 0.2684    
Epoch 9/10
20000/20000 [==============================] - 44s - loss: 0.2687    
Epoch 10/10
20000/20000 [==============================] - 44s - loss: 0.2713    


In [72]:
%%time
%time yp = model.predict(X)
%time yp_binary = get_binary_0_5(yp)
#print yvp
info('Generating Training Metrics')
training_metrics = get_metrics(y, yp, yp_binary)
print "****** Training Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
    training_metrics['coverage_error'], training_metrics['top_3'], training_metrics['top_5'], 
    training_metrics['f1_micro'], training_metrics['f1_macro'])

2017-03-07 15:30:31,828 : INFO : Generating Training Metrics


CPU times: user 38.9 s, sys: 17.3 s, total: 56.2 s
Wall time: 56.2 s
CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 23.7 ms
****** Validation Metrics: Cov Err: 2.181 | Top 3: 0.853 | Top 5: 0.954 | F1 Micro: 0.541 | F1 Macro: 0.403
CPU times: user 39.3 s, sys: 17.3 s, total: 56.7 s
Wall time: 56.6 s


In [73]:
model.fit(x=X, y=y, batch_size=NN_BATCH_SIZE, \
                              nb_epoch=20, verbose=MODEL_VERBOSITY)

Epoch 1/20
20000/20000 [==============================] - 44s - loss: 0.2643    
Epoch 2/20
20000/20000 [==============================] - 44s - loss: 0.2611    
Epoch 3/20
20000/20000 [==============================] - 44s - loss: 0.2612    
Epoch 4/20
20000/20000 [==============================] - 44s - loss: 0.2650    
Epoch 5/20
20000/20000 [==============================] - 44s - loss: 0.2597    
Epoch 6/20
20000/20000 [==============================] - 44s - loss: 0.2572    
Epoch 7/20
20000/20000 [==============================] - 44s - loss: 0.2551    
Epoch 8/20
20000/20000 [==============================] - 44s - loss: 0.2519    
Epoch 9/20
20000/20000 [==============================] - 44s - loss: 0.2484    
Epoch 10/20
20000/20000 [==============================] - 44s - loss: 0.2497    
Epoch 11/20
20000/20000 [==============================] - 44s - loss: 0.2501    
Epoch 12/20
20000/20000 [==============================] - 44s - loss: 0.2461    
Epoch 13/20
20000/20000 [

In [74]:
%%time
%time yp = model.predict(X)
%time yp_binary = get_binary_0_5(yp)
#print yvp
info('Generating Training Metrics')
training_metrics = get_metrics(y, yp, yp_binary)
print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
    training_metrics['coverage_error'], training_metrics['top_3'], training_metrics['top_5'], 
    training_metrics['f1_micro'], training_metrics['f1_macro'])

2017-03-07 16:05:49,006 : INFO : Generating Training Metrics


CPU times: user 38 s, sys: 18.4 s, total: 56.4 s
Wall time: 56.4 s
CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 23.7 ms
****** Validation Metrics: Cov Err: 1.980 | Top 3: 0.887 | Top 5: 0.963 | F1 Micro: 0.628 | F1 Macro: 0.467
CPU times: user 38.4 s, sys: 18.4 s, total: 56.9 s
Wall time: 56.9 s


In [ ]:
%%time
# when resuming, resume from an epoch with a previously created doc2vec model to get the learning rate right
start_from = 1
for epoch in range(start_from, DOC2VEC_MAX_EPOCHS+1):
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    info("****************** Epoch {} --- Working on {} *******************".format(epoch, GLOBAL_VARS.MODEL_NAME))
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        doc2vec_model.workers = NUM_CORES
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        # train the doc2vec model
        training_docs_iterator = ExtendedPVDocumentBatchGenerator(training_preprocessed_files_prefix, batch_size=10000)
        doc2vec_model.train(sentences=training_docs_iterator, report_delay=REPORT_DELAY)
        doc2vec_model.alpha -= DOC2VEC_ALPHA_DECREASE  # decrease the learning rate
        doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
        ensure_disk_location_exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME))
        doc2vec_model.save(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
        
        # get the word2vec analogy accuracy score
#         word2vec_result = doc2vec_model.accuracy(word2vec_questions_file, restrict_vocab=None)
#         epoch_word2vec_metrics.append(word2vec_result)
#         pickle.dump(word2vec_result, open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME,
#                                                        WORD2VEC_METRICS_FILENAME), 'w'))

    # Validation Embeddings
    info('Getting Validation Embeddings')
    Xv, yv = get_extended_docs_with_inference(doc2vec_model, doc_classification_map, classifications, 
                                     validation_docs_list, VALIDATION_MATRIX, 
                                     validation_preprocessed_files_prefix)

2017-03-06 01:24:19,997 : INFO : ****************** Epoch 1 --- Working on doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_1 *******************
2017-03-06 01:24:20,042 : INFO : loading Doc2Vec object from /big/s/shalaby/parameter_search_doc2vec_models_extended/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_1/model
